In [12]:
import pandas as pd
import numpy as np

df_knn = pd.read_csv('KNN_resultados.csv')
df_xgb = pd.read_csv('XGBoost_resultados.csv')

In [13]:
df_knn = df_knn.drop(columns=["Unnamed: 0", "Train RMSE", "Test RMSE", "Overfitting?", "Underfitting?"])
df_knn = df_knn.rename(columns={
    'Real':'Valores Reais',
    'Previsto':'KNN_Previsto',
    'Erro Absoluto':'KNN_MAE'
        })
df_knn

,Empresa,Data,Valores Reais,KNN_Previsto,KNN_MAE
0,3tentos,2022-10-01,0.000718,0.009481,0.008763
1,3tentos,2023-01-01,-0.002094,0.005192,0.007286
2,3tentos,2023-04-01,-0.001256,0.000200,0.001456
3,3tentos,2023-07-01,0.003401,0.001250,0.002151
4,3tentos,2023-10-01,-0.004247,0.000192,0.004440
...,...,...,...,...,...
7921,Zamp S.A.,2023-07-01,0.002731,-0.000838,0.003569
7922,Zamp S.A.,2023-10-01,0.006019,-0.000071,0.006090
7923,Zamp S.A.,2024-01-01,-0.018266,-0.005160,0.013106
7924,Zamp S.A.,2024-04-01,0.013895,0.005300,0.008595


In [14]:
df_xgb = df_xgb.drop(columns=["Unnamed: 0", "Train RMSE", "Test RMSE", "Overfitting?", "Underfitting?"])
df_xgb = df_xgb.rename(columns={
    'Real':'Valores Reais',
    'Previsto':'xgb_Previsto',
    'Erro Absoluto':'xgb_MAE'
        })
df_xgb

,Empresa,Data,Valores Reais,xgb_Previsto,xgb_MAE
0,3tentos,2022-10-01,0.000718,0.009481,0.008763
1,3tentos,2023-01-01,-0.002094,0.007728,0.009823
2,3tentos,2023-04-01,-0.001256,0.006091,0.007348
3,3tentos,2023-07-01,0.003401,0.005042,0.001641
4,3tentos,2023-10-01,-0.004247,0.004837,0.009084
...,...,...,...,...,...
7921,Zamp S.A.,2023-07-01,0.002731,0.001208,0.001523
7922,Zamp S.A.,2023-10-01,0.006019,0.001271,0.004748
7923,Zamp S.A.,2024-01-01,-0.018266,0.001461,0.019727
7924,Zamp S.A.,2024-04-01,0.013895,0.000702,0.013193


In [15]:
df = pd.merge(df_knn, df_xgb, on=['Empresa', 'Data', 'Valores Reais'], how='right')
df

,Empresa,Data,Valores Reais,KNN_Previsto,KNN_MAE,xgb_Previsto,xgb_MAE
0,3tentos,2022-10-01,0.000718,0.009481,0.008763,0.009481,0.008763
1,3tentos,2023-01-01,-0.002094,0.005192,0.007286,0.007728,0.009823
2,3tentos,2023-04-01,-0.001256,0.000200,0.001456,0.006091,0.007348
3,3tentos,2023-07-01,0.003401,0.001250,0.002151,0.005042,0.001641
4,3tentos,2023-10-01,-0.004247,0.000192,0.004440,0.004837,0.009084
...,...,...,...,...,...,...,...
7921,Zamp S.A.,2023-07-01,0.002731,-0.000838,0.003569,0.001208,0.001523
7922,Zamp S.A.,2023-10-01,0.006019,-0.000071,0.006090,0.001271,0.004748
7923,Zamp S.A.,2024-01-01,-0.018266,-0.005160,0.013106,0.001461,0.019727
7924,Zamp S.A.,2024-04-01,0.013895,0.005300,0.008595,0.000702,0.013193


In [16]:
grupo = df.groupby('Empresa')[['KNN_MAE', 'xgb_MAE']].mean().reset_index()
grupo

,Empresa,KNN_MAE,xgb_MAE
0,3tentos,0.005229,0.006244
1,Aco Altona,0.000354,0.000361
2,Aeris,0.003211,0.004010
3,Afluente T,0.000388,0.000364
4,Agrogalaxy,0.008917,0.012790
...,...,...,...
275,Wilson Sons,0.001594,0.002507
276,Wiz Co,0.006316,0.006547
277,Wlm Ind Com,0.000201,0.000181
278,Yduqs Part,0.005485,0.008619


In [17]:
from scipy.stats import friedmanchisquare
from scipy.stats import rankdata

friedmanchisquare(grupo['KNN_MAE'], grupo['xgb_MAE']) #faltando 1 modelo

# Aplicar ranking por linha (menor MAE = melhor)
ranks = grupo[['KNN_MAE', 'xgb_MAE']].apply(rankdata, axis=1)
mean_ranks = ranks.mean(axis=0)

# Exibir ranking médio
print("Rank médio dos modelos:")
for i, rank in enumerate(mean_ranks, start=1):
    print(f"Modelo {i}: {rank:.3f}")

ValueError: At least 3 sets of samples must be given for Friedman test, got 2.